In [146]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline
font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc("font",family=font_name)

In [147]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import mglearn

In [148]:
titanic_train = pd.read_csv('../data/titanic/train.csv')
titanic_test = pd.read_csv('../data/titanic/test.csv')
titanic_test_result = pd.read_csv('../data/titanic/gender_submission.csv')

In [149]:
titanic_test = pd.merge(titanic_test, titanic_test_result, how='outer')

In [150]:
def titanic_data(raw_data):
    tmp = []
    for each in raw_data['Sex']:
        if each == 'female':
            tmp.append(0)
        elif each == 'male':
            tmp.append(1)
        else:
            tmp.append(np.nan)
            
    tmp_age = []
    for each in raw_data['Age']:
        if each < 10:
            tmp_age.append(0)
        elif (10 <=each) & (each<20) :
            tmp_age.append(1)
        elif (20<=each) & (each<30 ):
            tmp_age.append(2)
        elif (30 <=each) & (each<40) :
            tmp_age.append(3)
        elif (40 <=each) & (each<50) :
            tmp_age.append(4)
        elif (50 <=each) & (each<60) :
            tmp_age.append(6)
        elif (60 <=each) & (each<70) :
            tmp_age.append(7)
            
        elif (70 <=each) & (each<=80) :
            tmp_age.append(8)
        else:
            tmp_age.append(np.nan)
            
    
    raw_data['sex'] = tmp
    raw_data['Age'] = tmp_age
    raw_data['Survived'] = raw_data['Survived'].astype('float')
    raw_data['Pclass'] = raw_data['Pclass'].astype('float')
    raw_data['sex'] = raw_data['sex'].astype('float')
    raw_data['SibSp'] = raw_data['SibSp'].astype('float')
    raw_data['Parch'] = raw_data['Parch'].astype('float')
    raw_data['Fare'] = raw_data['Fare'].astype('float')
    raw_data['Age'] = raw_data['Age'].astype('float')
    return raw_data

In [151]:
titanic_train_data = titanic_data(titanic_train)
titanic_test_data = titanic_data(titanic_test)

In [152]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

y_train  = titanic_train_data['Survived'].astype(float)
X_train = titanic_train_data[['Pclass', 'sex', 'Age', 'SibSp', 'Parch', 'Fare']]
X_train.isnull().sum()
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())

standard_scaler.fit(X_train)
X_train_scaled_standard = standard_scaler.transform(X_train)


C:\Users\A\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [153]:
y_test = titanic_test_data['Survived'].astype(float)
X_test = titanic_test_data[['Pclass', 'sex', 'Age', 'SibSp', 'Parch', 'Fare']]
X_test.isnull().sum()
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())
X_test_scaled_standard = standard_scaler.transform(X_test)

C:\Users\A\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\A\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


# DecisionTreeClassifier

In [173]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=0)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [174]:
print('기본 titanic 정제 데이터 점수:')
print('Train Score:{}'.format(tree_clf.score(X_train, y_train)))
print('Test Score: {}'.format(tree_clf.score(X_test, y_test)))

기본 titanic 정제 데이터 점수:
Train Score:0.8305274971941639
Test Score: 0.9641148325358851


In [184]:
result = tree_clf.predict(X_test)
output = pd.DataFrame({'PassengerId' : titanic_test_data['PassengerId'],
                      'Survived' : result})
output['Survived'] = output['Survived'].astype(int)
output.to_csv('my_submission_decision_tree.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# 랜덤 포레스트

In [178]:
estimators = []
trainScore =[]
TestScore = []
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm_notebook
for i in tqdm_notebook(range(1,200)):
    forest = RandomForestClassifier(n_estimators = i, random_state = 0)
    forest.fit(X_train, y_train)
    estimators.append(i)
    trainScore.append(format(forest.score(X_train, y_train)))
    TestScore.append(format(forest.score(X_test, y_test)))
    
all_forest = pd.DataFrame({'n_estimators' : estimators,
                          'trainScore' : trainScore,
                          'TestScore': TestScore})
all_forest['trainScore'] = all_forest['trainScore'].astype(float)
all_forest['TestScore'] = all_forest['TestScore'].astype(float)
all_forest[all_forest['TestScore'] == all_forest['TestScore'].max()]            

,n_estimators,trainScore,TestScore
19,20,0.946128,0.858852


In [182]:
forest = RandomForestClassifier(n_estimators = 20, random_state = 0)
forest.fit(X_train, y_train)
result = forest.predict(X_test)
output = pd.DataFrame({'PassengerId' : titanic_test_data['PassengerId'],
                      'Survived' : result})
output['Survived'] = output['Survived'].astype(int)
output.to_csv('my_submission_RandomForestClassifier', index=False)
output.head()

,PassengerId,Survived
0,892,1
1,893,0
2,894,0
3,895,0
4,896,0


# 그레이디언트 부스트 회귀 트리

In [206]:
estimators = []
trainScore =[]
TestScore = []
from tqdm import tqdm_notebook
from sklearn.ensemble import GradientBoostingClassifier
for i in tqdm_notebook(range(1,50)):
    gbrt = GradientBoostingClassifier(n_estimators= i, random_state= 0)
    gbrt.fit(X_train, y_train)
    estimators.append(i)
    trainScore.append(format(gbrt.score(X_train, y_train)))
    TestScore.append(format(gbrt.score(X_test, y_test)))
all_Gradient = pd.DataFrame({'n_estimators' : estimators,
                          'trainScore' : trainScore,
                          'TestScore': TestScore})
all_Gradient['trainScore'] = all_Gradient['trainScore'].astype(float)
all_Gradient['TestScore'] = all_Gradient['TestScore'].astype(float)
all_Gradient[all_Gradient['TestScore'] == all_Gradient['TestScore'].max()]

,n_estimators,trainScore,TestScore
13,14,0.833895,0.961722
18,19,0.837262,0.961722
19,20,0.837262,0.961722
20,21,0.837262,0.961722


In [207]:
gbrt = GradientBoostingClassifier(n_estimators= 14, random_state= 0)
gbrt.fit(X_train, y_train)
result = gbrt.predict(X_test)
output = pd.DataFrame({'PassengerId' : titanic_test_data['PassengerId'],
                      'Survived' : result})
output['Survived'] = output['Survived'].astype(int)
output.to_csv('my_submission_Gradient.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# 배깅

In [204]:
estimators = []
trainScore =[]
TestScore = []
from tqdm import tqdm_notebook
from sklearn.ensemble import BaggingClassifier
for i in tqdm_notebook(range(1,2)):
    bagg = BaggingClassifier( DecisionTreeClassifier(),n_estimators= 125, random_state = 0)
    bagg.fit(X_train_scaled_standard, y_train)
    estimators.append(i)
    trainScore.append(format(bagg.score(X_train_scaled_standard, y_train)))
    TestScore.append(format(bagg.score(X_test_scaled_standard, y_test)))
    


In [205]:
all_bagg = pd.DataFrame({'Max_depth' : estimators,
                          'trainScore' : trainScore,
                          'TestScore': TestScore})
all_bagg['trainScore'] = all_bagg['trainScore'].astype(float)
all_bagg['TestScore'] = all_bagg['TestScore'].astype(float)
all_bagg[all_bagg['TestScore'] == all_bagg['TestScore'].max()]


,Max_depth,trainScore,TestScore
0,1,0.95174,0.863636
